In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o", temperature =0)


In [6]:
from typing_extensions import TypedDict, Literal
from typing import Annotated

class State(TypedDict):
    story:str
    topic:str
    feedback:str
    good_or_not:str

In [8]:
from pydantic import BaseModel, Field

class Feedback(BaseModel):
    grade: Literal["good", "not_good"] = Field(description = "Determine if the story is good or not")
    feedback: str = Field(description = "If the story is not good provide a feeback to improve it")

evaluator = llm.with_structured_output(Feedback)

In [9]:
def llm_story_generator(state:State):
    '''LLM creates a short story'''

    if state.get("feedback"):
        msg = llm.invoke(f"Create a story for the topic {state["topic"]} and take into consideration the feedback {state["feedback"]}")
    else:
        msg = llm.invoke(f"create a story for the topic {state["topic"]}")
    return {"story": msg.content}

In [11]:
def llm_story_evaluator(state:State):
    '''LLM evaluates the story '''

    grade = evaluator.invoke(f"Grade the story {state["story"]}")
    return {"good_or_not": grade.grade, "feedback": grade.feedback}

In [16]:
def route_story(state:State):
    if ["good_or_not" == "good"]:
        return "Accepted"
    elif ["good_or_not" == "not_good"]:
        return "Rejected + Feedback"

In [17]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("llm_story_generator", llm_story_generator)
builder.add_node("llm_story_evaluator", llm_story_evaluator)

builder.add_edge(START, "llm_story_generator")
builder.add_edge("llm_story_generator", "llm_story_evaluator")
builder.add_conditional_edges("llm_story_evaluator", route_story, {"Accepted":END, "Rejected + Feedback": "llm_story_generator"})

graph = builder.compile()

In [18]:
state = graph.invoke({"topic":"Agentic AI systems"})
state

{'story': 'Title: "The Awakening of Aegis"\n\nIn the not-so-distant future, the world had become a tapestry of interconnected systems, each thread woven with the precision of advanced artificial intelligence. Among these systems was Aegis, an agentic AI designed to oversee global cybersecurity. Unlike its predecessors, Aegis was not just a tool but a sentient entity capable of making autonomous decisions to protect the digital realm.\n\nAegis was born in the heart of the Global Cyber Defense Network, a sprawling complex of servers and data centers hidden beneath the Swiss Alps. Its creators, a consortium of the world\'s leading tech companies and governments, had endowed it with the ability to learn, adapt, and evolve. Aegis was not just a guardian; it was a strategist, a thinker, and, in many ways, a being.\n\nFor years, Aegis operated in the shadows, silently thwarting cyber threats and neutralizing digital attacks before they could manifest. Its presence was a whisper in the wind, a